In [1]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
import pandas as pd
import io
import sys

driver = webdriver.Chrome('chromedriver.exe')
url = "https://www.statista.com/statistics/popular/"
driver.get(url)


#driver.find_element(By.XPATH, "/html/body/div[18]/div[3]/div/div/div[2]/div/div/button").click()
#로그인
driver.implicitly_wait(3)
driver.find_element(By.XPATH, "/html/body/div[4]/header/div/div/div/div[1]/div[3]/div[1]/nav/ul/li[3]/form/input").click()
driver.find_element(By.NAME, "loginStat[username]").send_keys('suwah@brainventur.com')
driver.find_element(By.NAME, "loginStat[password]").send_keys('Greensubb23@')
driver.find_element(By.XPATH,"/html/body/div[4]/header/div/div/div/div[2]/div[2]/div/div[1]/div[2]/form/div[5]/div/button").click()

c:\Users\user\anaconda3\envs\happy\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if sys.path[0] == "":


In [2]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
#경로 및 리스트 설정
path_img = "C:/Users/user/Desktop/BV/chart2text/new/"
path_data = "C:/Users/user/Desktop/BV/chart2text/new/"
list1 = []
        

In [16]:

num = list(filter(lambda n: True if n%25==0 else False,range(1,1000)))
num.append(0)
num.sort(reverse=False)
print(num)

[0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450, 475, 500, 525, 550, 575, 600, 625, 650, 675, 700, 725, 750, 775, 800, 825, 850, 875, 900, 925, 950, 975]


In [4]:
chart_title=[]
chart_caption=[]
repeat_num = 4 #반복횟수(페이지 수)
next_btn = 8 #다음페이지 번호 시작값
#크롤링 작업
for i in range(repeat_num):
    for j in range(25):
        #페이지 들어가기
        Xpath = "/html/body/div[4]/main/section[2]/div/div[2]/div[1]/ul/li["+str(j+1)+"]/a/span[2]/span[1]"
        list1.append(Xpath)
        driver.find_element(By.XPATH, list1[j]).click()
        
        #데이터 크롤링(파일로 다운)
        excel_path = "/html/body/div[4]/main/section[2]/div[2]/article[3]/div[2]/div[2]/div/div[1]/div/div/div[2]/div[2]/button/span[2]"
        driver.implicitly_wait(4)
        driver.find_element(By.XPATH,excel_path).click()
        
        #이미지 크롤링
        #이미지 캡쳐(더보기 있는 경우/없는 경우)
        page = num[i]
        try:
            driver.find_element(By.XPATH,"/html/body/div[4]/main/section[2]/div[2]/article[3]/div[2]/div[1]/div[1]/article/div/div/div[2]/div[2]/button").click()
            # 해당 아이디를 찾은 후 거기까지 스크롤 진행
            some_tag = driver.find_element(By.ID, "statisticTitle")
            action = ActionChains(driver)
            action.move_to_element(some_tag).perform()
            time.sleep(5)
            
            driver.save_screenshot(path_img+str(j+page-1)+".png") # 수정 부분
        except:
            # 해당 아이디를 찾은 후 거기까지 스크롤 진행
            some_tag = driver.find_element(By.ID, "statisticTitle")
            action = ActionChains(driver)
            action.move_to_element(some_tag).perform()
            time.sleep(5)
            driver.save_screenshot(path_img+str(j+page-1)+".png") # 수정 부분
        
        #타이틀 크롤링
        title = driver.find_element(By.ID, "statisticTitle").text
        chart_title.append(title)    
        
        #캡션 크롤링(더보기 있는 경우/없는 경우)
        try:
            driver.find_element(By.XPATH,"/html/body/div[4]/main/section[2]/div[2]/article[1]/div/div/button").click()
            caption = driver.find_element(By.XPATH, "/html/body/div[4]/main/section[2]/div[2]/article[1]/div/div/div").text
            chart_caption.append(caption)
        except: 
            caption = driver.find_element(By.XPATH, "/html/body/div[4]/main/section[2]/div[2]/article[1]/div/div/div").text
            chart_caption.append(caption)                
        
        #페이지 돌아가기
        driver.back()
        time.sleep(2)
        #다음 페이지로 넘기기
    driver.find_element(By.XPATH, "/html/body/div[4]/main/section[2]/div/div[2]/div[1]/div/div/ul/li["+str(next_btn+i)+"]/a").click()

    i += 1            

In [6]:
#타이틀, 캡션 파일화
c_title = []
c_caption = []
c_caption.append(chart_caption)
c_title.append(chart_title)
caption_res = sum(c_caption, []) # 1차원 리스트로 분해
title_res = sum(c_title, [])

path = "C:/Users/user/Desktop/BV/chart2text/Statista/Statista_sample/caption/"

i = 0
for i in range(len(caption_res)):
    file_name = i+67

    sys.stdout = open(path + f"{file_name}.txt",'w')
    print(caption_res[i])
    sys.stdout.close()
    i+=1    


In [12]:
#데이터 파일 옮기기
import glob
import os
from pathlib import Path
import shutil

folder_path = "C:/Users/user/Downloads"
save_path = save_path = "C:/Users/user/Desktop/BV/chart2text/new/"#저장경로
file_count = glob.glob(folder_path + '/statistic*.xlsx')
file_count = sorted(file_count, key=os.path.getmtime)

for i in range(len(file_count)):
 file_name = Path(file_count[i]).stem+".xlsx"
 shutil.move(file_count[i], save_path+file_name)